In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from models.gnn_train import train_gnn, get_gnn_trained_embedding
import os

e:\anaconda\envs\coreferee_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
test_path = os.path.join("..","data", "multinews", "test.jsonl")
train_gnn(
     file_path = test_path,
     hidden_size=128,
     out_size=256,
     num_heads=3,
     learning_rate=0.001,
     num_epochs=2,
     batch_size=2
)

data size: <bound method SummaryDataset.__len__ of <models.DatasetLoader.SummaryDataset object at 0x00000206A44C86D0>>
hidden_size:128, out_size: 256, num_heads: 3, sentence_in_size: 768, word_in_size: 768, feat_drop: 0.2, attn_drop: 0.2
Initial sentence_feat: tensor([[-0.0106, -0.0281, -0.0433,  ..., -0.0406,  0.0261,  0.0451],
        [ 0.0117, -0.0213, -0.0567,  ..., -0.0164,  0.0444,  0.0274],
        [-0.0309, -0.0361, -0.0391,  ..., -0.0118,  0.0344, -0.0021],
        ...,
        [-0.0304, -0.0248, -0.0056,  ..., -0.0141, -0.0391,  0.0263],
        [-0.0316, -0.0353, -0.0104,  ..., -0.0025,  0.0190, -0.0063],
        [-0.0047, -0.0168,  0.0017,  ..., -0.0401,  0.0083,  0.0074]])
Initial word_feat: tensor([[-0.0071, -0.0108, -0.0136,  ..., -0.0095, -0.0129, -0.0297],
        [-0.0180, -0.0373, -0.0034,  ...,  0.0090, -0.0046, -0.0020],
        [ 0.0132, -0.0273, -0.0112,  ..., -0.0034,  0.0024, -0.0186],
        [ 0.0120, -0.0411, -0.0159,  ..., -0.0181, -0.0033, -0.0133],
      

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


: 